In [8]:
from platform import python_version

print(python_version())

3.10.12


In [9]:
import pyreadr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import numpy as np
import scipy.stats as stats
from sklearn.decomposition import PCA
from scipy.linalg import eigh
from sklearn.preprocessing import StandardScaler

from fault_detection_algorithms.PCAFaultDetector import PCAFaultDetector
from fault_detection_algorithms.PLSFaultDetector import PLSFaultDetector 

from fast_dataset_open import open_with_cache

In [90]:

import importlib
import fault_detection_algorithms.PCAFaultDetector
import fault_detection_algorithms.PLSFaultDetector
import fault_detection_algorithms.fault_detector

importlib.reload(fault_detection_algorithms.PCAFaultDetector)
importlib.reload(fault_detection_algorithms.PLSFaultDetector)
importlib.reload(fault_detection_algorithms.fault_detector)

from fault_detection_algorithms.PCAFaultDetector import PCAFaultDetector
from fault_detection_algorithms.PLSFaultDetector import PLSFaultDetector 
from fault_detection_algorithms.fault_detector import BaseFaultDetectionAlgorithm

In [11]:
# constants
dataset_dir = "./datasets/TEP/"

In [12]:
X_columns= [ 'xmeas_1', 'xmeas_2', 'xmeas_3', 'xmeas_4', 'xmeas_5', 'xmeas_6', 'xmeas_7', 'xmeas_8', 'xmeas_9', 'xmeas_10', 'xmeas_11', 'xmeas_12', 'xmeas_13', 'xmeas_14', 'xmeas_15', 'xmeas_16', 'xmeas_17', 'xmeas_18', 'xmeas_19', 'xmeas_20', 'xmeas_21', 'xmeas_22', 'xmv_1', 'xmv_2', 'xmv_3', 'xmv_4', 'xmv_5', 'xmv_6', 'xmv_7', 'xmv_8', 'xmv_9', 'xmv_10', 'xmv_11']

In [13]:
training_fault_free_df = open_with_cache(f"{dataset_dir}/TEP_FaultFree_Training.RData")

Loading from cache: cache_data/40501dd3f9438f820d63e297020b0214.pkl


In [14]:
X= training_fault_free_df[X_columns].values

In [15]:
Y= training_fault_free_df["xmeas_35"].values

In [16]:
Y.shape

(250000,)

In [91]:
fault_detector_pca = PCAFaultDetector()
fault_detector_pca_with_resi_scale = PCAFaultDetector(scale_residuals=True)
fault_detector_pls = PLSFaultDetector(n_components=29)

In [92]:
fault_detector_pca.train(X)
fault_detector_pca_with_resi_scale.train(X)
fault_detector_pls.train(X, Y.reshape(-1,1))


In [93]:
fault_detector_pca.save("models/pca_fault_detector.pickle")
fault_detector_pca_with_resi_scale.save("models/pca_fault_detector_with_resi_scale.pickle")
fault_detector_pls.save("models/pls_fault_detector.pickle")

In [86]:
fault_detector_pca.J_th_SPE

14.205727480705509

In [88]:
fault_detector_pca_with_resi_scale.thresholds

[(1.8915609680000627, 0.0),
 (5.990175424499817, 6.41889062400736),
 (6.8112903161605365, 7.268421751513869),
 (7.350435178996791, 7.847550697738826),
 (7.783696840849631, 8.304383456922906),
 (8.140394680728782, 8.68967831286697),
 (8.471442255816582, 9.027465193451105),
 (8.75649533416951, 9.331185672238485),
 (9.016177183893443, 9.609185344889493),
 (9.25792729870849, 9.867032950443075),
 (9.486430579062521, 10.108646544506811),
 (9.69672340639493, 10.33689740111278),
 (9.903011139474657, 10.553958894111366),
 (10.09621270584849, 10.761520127538255),
 (10.281468555946415, 10.960923017347769),
 (10.464450278009675, 11.153253710187432),
 (10.641762729074262, 11.33940555139636),
 (10.815865851479522, 11.520123666344746),
 (10.980116343638826, 11.696037281419688),
 (11.14238878767989, 11.867683645986304),
 (11.296953391730504, 12.035526063314904),
 (11.453374273767606, 12.199967702790344),
 (11.60970149441845, 12.36136233485639),
 (11.757130892265995, 12.520022783855048),
 (11.904896567

In [52]:
fault_detector_pca.P_res @ fault_detector_pca.P_res.T

array([[ 5.24183107e-01,  2.58854369e-02,  2.45912861e-02, ...,
        -2.55036942e-02,  1.38578546e-02, -5.91659928e-05],
       [ 2.58854369e-02,  6.63225724e-01,  4.92703433e-03, ...,
         2.27732690e-02, -4.25736631e-02,  6.07704334e-03],
       [ 2.45912861e-02,  4.92703433e-03,  3.63431659e-02, ...,
        -2.02111704e-02,  9.37819478e-04,  2.89212081e-03],
       ...,
       [-2.55036942e-02,  2.27732690e-02, -2.02111704e-02, ...,
         6.89554169e-01, -3.21430243e-02,  2.17799828e-04],
       [ 1.38578546e-02, -4.25736631e-02,  9.37819478e-04, ...,
        -3.21430243e-02,  5.38389110e-01, -1.42470853e-03],
       [-5.91659928e-05,  6.07704334e-03,  2.89212081e-03, ...,
         2.17799828e-04, -1.42470853e-03,  5.01916134e-01]])

In [53]:
x = X[0]


In [30]:
p_r = fault_detector_pca.P_res
resi = fault_detector_pca.residual_eigenvalues

In [69]:
(x@p_r@p_r.T@x.T)
(x@p_r@np.linalg.inv(np.diag(resi) )@p_r.T@x.T)

1060523346.6933136

In [72]:
np.sum((x@p_r)**2/resi)

1060523346.693316